## What can we do with DFT? - Exercises
In this notebook, you find some exercises related with what can be done using DFT

For the first 3 exercises, we recommend using LCAO mode for speed, so you can use the following settings for your calculator object in all of the exercises.

```python
calc = GPAW(h=0.24,
            mode='lcao',
            basis='sz(dzp)',
            xc='PBE')
```
The Single-$\zeta$ (sz) basis set is not a very good basis set, and is generally not used for real calculations. However, for our purposes, and to speed up the calculations, this will be sufficient. For a real simulation, one would generally use a double-$\zeta$ polarized (dzp) basis set at least. The `sz(dzp)` basis means, that we use the `sz` sub-basis from the larger `dzp` basis set.

For relaxations we recommend the `BFGS` method.

## Exercise 1
Calculate the relaxed geometry of ammonia, NH$_3$, and its atomization energy with respect to H$_2$ and N$_2$, and with respect to isolated H and N.

See the following GPAW tutorial for inspiration: https://wiki.fysik.dtu.dk/gpaw/tutorials/H2/atomization.html

For building molecules, refer to the ASE wiki: https://wiki.fysik.dtu.dk/ase/ase/build/build.html#molecules

Questions: 
* Why do we use N2 and H2 as references for the atomization energy?
* What shall we include in the calculator object if we want to use N and H as reference systems?
* How much does the atomization energy changes if we use N and H as reference systems instead of N2 and H2?

In [ ]:
%%writefile ex1.py

In [ ]:
!qsub.py -t 1 -p 8 ex1.py

In [ ]:
!qstat -u $USER

## Exercise 2
Calculate the vibrational modes of ammonia.

See https://wiki.fysik.dtu.dk/ase/ase/vibrations/vibrations.html

Questions:
* Is the highest frequency mode of ammonia larger than the one of water?
* What is the shape of the highest vibrational mode of ammonia?

In [ ]:
#%%writefile ex2.py
# student
from ase.io import read

atoms = read(?)  # Load the relaxed ammonia from the previous exercise
calc = ?
atoms.set_calculator(calc)

# Create vibration calculator
vib = ?

# Make trajectory files to visualize normal modes:
for mode in range(3*len(atoms)):  # 3N vibrational modes, where N is the number of atoms
    vib.write_mode(mode)

In [ ]:
# Visualize the modes we just made
# Change the number to a mode between 0-11
!ase gui vib.0.traj

In [ ]:
!qsub.py -t 1 -p 8 ex2.py

In [ ]:
!qstat -u $USER

## Exercise 3
Set up a 2x2x3 Pt(111) surface with the two bottom layers constraint and calculate the adsorption energy of H on its hollow, bridge and top sites. You can reuse the energies from exercise 1 for the energy of an H$_2$ molecule.

Use a (2x2x1) k-point sampling for your surface.

See
https://wiki.fysik.dtu.dk/ase/ase/build/surface.html

https://wiki.fysik.dtu.dk/ase/ase/build/surface.html#ase.build.add_adsorbate

**Important:** Remember to always visualize your structures!

Questions:
* What is the most stable adsorption site on Pt111?
* When do we need to include k-points in a calculator?
* Do you think that a (2x2x1) k-point mesh is a sufficient number for a good simulation? Why?

In [ ]:
#%%writefile ex3.py
# student
from ase.constraints import FixAtoms
from ase.build import fcc111, add_adsorbate

slab = fcc111(?)  # Construct your pt111 (2x2x3) surface
# Constrain everything but the top layer
c = FixAtoms(indices=[atom.index for atom in slab if atom.tag > 1])
slab.set_constraint(c)

# Relax the clean surface first

# Adding adsorbates
position = 'fcc'  # Specify site
add_adsorbate(slab, 'H', 1.5, position)

In [ ]:
# This calculation might take a while, so we use more processors
!qsub.py -t 2 -p 32 ex3.py

In [ ]:
!qstat -u $USER

## Exercise 4

For this exercise, we need to switch from LCAO mode to Plane-Wave (PW) mode, as we only have the stress-tensor in PW mode.

Optimize NaCl using the stress tensor (check convergence of k-points and PW mode cutoff before running the optimization). Remember to save the gpw file of the relaxed configuration, because you will need it later. Saving the gpw file is done from the calculator, see https://wiki.fysik.dtu.dk/gpaw/tutorials/plotting/plot_wave_functions.html#creating-a-wave-function-file

See https://wiki.fysik.dtu.dk/gpaw/exercises/stress/stress.html on doing Plane-Wave calculations. Also see https://wiki.fysik.dtu.dk/ase/ase/constraints.html#ase.constraints.UnitCellFilter on relaxing using the stress tensor.

Use a (NxNxN) k-point sampling for the NaCl cell.

Questions:
* What is the relaxed lattice parameter for NaCl?

In [ ]:
#%%writefile ex4a.py
# student
from ase.spacegroup import crystal

# Set up a NaCl unit cell
a = 5.64
nacl = crystal(['Na', 'Cl'], [(0, 0, 0), (0.5, 0.5, 0.5)], spacegroup=225, cellpar=[a, a, a, 90, 90, 90])

In [ ]:
!qsub.py -p 8 -t 1 ex4a.py

In [ ]:
!qstat -u $USER

## Exercise 5
Calculate the band gap and the band structure of NaCl along the path $\Gamma$XM. You can reuse the gpw file from the previous exercise to speed up the calculations.

See https://wiki.fysik.dtu.dk/gpaw/tutorials/bandstructures/bandstructures.html on band structures. Include at least 5 unoccupied bands.

Tip: The $\Gamma$ k-point in ASE & GPAW is called G

Your band structure should look something like the following:

![Band_structure](ans_bs.png)

Questions:
* Why does DFT underestimate the band gap?
* What is the band gap of NaCl?
* Where are the valence and conduction band edges for NaCl located?

## Exercise 6
Calculate the Density of States and the Projected Density of States (on Na and Cl) of NaCl

See https://wiki.fysik.dtu.dk/ase/ase/dft/dos.html and https://wiki.fysik.dtu.dk/gpaw/documentation/pdos/pdos.html#atomic-orbital-pdos for doing DOS and PDOS.

Questions:
* What is the difference between DOS and PDOS?
* Which elements composes the band edges of NaCl?

## Exercise 7
Calculate the work function of the clean slab of Pt111 and of the most stable configuration of H on Pt111.

See https://wiki.fysik.dtu.dk/gpaw/exercises/surface/surface.html#work-function

Questions:
* What is the work function of Pt111?
* How does H affect the work function of Pt111, and why?